In [51]:
import os
import numpy as np
import random
from math import log, ceil

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [104]:
pad_chr = chr(0)
start_chr = chr(1)
end_chr = chr(2)

In [13]:
f = open("fra-eng/fra.txt", "r")

In [14]:
lines = [line.strip("\n").split("\t") for line in f]

In [15]:
lang1 = [x[0] for x in lines]
lang2 = [x[1] for x in lines]

In [115]:
def get_batch(lang1, lang2, alphabet1, alphabet2, num_samples):
    sample = random.sample(range(len(lang1)), num_samples)
    lang1_sample = [lang1[i] + end_chr for i in sample]
    lang2_in_sample = [start_chr + lang2[i] for i in sample]
    lang2_out_sample = [lang2[i] + end_chr for i in sample]
    maxlen1, maxlen2 = max(map(len, lang1_sample)), max(map(len, lang2_in_sample))
    lang1_sample = [[alphabet1[i] for i in x.ljust(maxlen1, pad_chr)] for x in lang1_sample]
    lang2_in_sample = [[alphabet2[i] for i in x.ljust(maxlen2, pad_chr)] for x in lang2_in_sample]
    lang2_out_sample = [[alphabet2[i] for i in x.ljust(maxlen2, pad_chr)] for x in lang2_out_sample]
    return np.asarray(lang1_sample), np.asarray(lang2_in_sample), np.asarray(lang2_out_sample)

In [116]:
def build_alphabet_converter(instances):
    chars = set.union(*map(set, instances))
    converter = dict(zip(chars, range(3, len(chars) + 3)))
    converter[pad_chr] = 0
    converter[start_chr] = 1
    converter[end_chr] = 2
    return converter

In [133]:
def build_alphabet_converter_2d(instances):
    chars = set.union(*map(set, instances))
    max_len = ceil(log(len(chars) + 3, 2))
    binary_strings = [list(map(int, bin(i)[2:].rjust(max_len, '0'))) for i in range(3, len(chars) + 3)]
    converter = dict(zip(chars, binary_strings))
    converter[pad_chr] = [0] * max_len
    
    start_val = [0] * (max_len - 1)
    start_val.append(1)
    converter[start_chr] = start_val
    
    end_val = [0] * (max_len - 2)
    end_val.extend([1, 0])
    converter[end_chr] = end_val
    
    return converter

In [134]:
alpha1 = build_alphabet_converter_2d(lang1)
alpha2 = build_alphabet_converter_2d(lang2)

In [138]:
x, y_in, y_out = get_batch(lang1, lang2, alpha1, alpha2, 100)
print(x.shape, y_in.shape, y_out.shape)

(100, 95, 7) (100, 91, 7) (100, 91, 7)
